<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/228_Cross_Sell_Upsell_Orchestrator_Tier2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LLM-powered rationale generation for Tier 2 Cross-Sell & Upsell Orchestrator

In [ ]:
"""LLM-powered rationale generation for Tier 2 Cross-Sell & Upsell Orchestrator"""

from typing import Dict, Any, Optional
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from config import Tier2CrossSellUpsellConfig


def generate_llm_rationale(
    opportunity: Dict[str, Any],
    customer_data: Dict[str, Any],
    product: Dict[str, Any],
    config: Tier2CrossSellUpsellConfig
) -> Optional[str]:
    """
    Generate a personalized, natural language rationale using LLM.

    This creates a more engaging and personalized explanation for why a product
    is recommended, based on the customer's profile, concerns, and routine.

    Args:
        opportunity: Opportunity dict with recommendation_type, rationale, scores
        customer_data: Customer data with name, skin_concerns, loyalty_tier, etc.
        product: Product dict with name, category, ingredients, target_concerns
        config: Configuration with LLM settings

    Returns:
        Enhanced rationale string, or None if LLM call fails (fallback to rule-based)
    """
    try:
        # Extract key information
        customer_name = customer_data.get("name", "the customer")
        customer_concerns = customer_data.get("skin_concerns", [])
        customer_tier = customer_data.get("loyalty_tier", "Unknown")
        customer_ltv = customer_data.get("lifetime_value", 0.0)

        product_name = product.get("name", "")
        product_category = product.get("category", "")
        product_price = product.get("price", 0.0)
        product_ingredients = product.get("active_ingredients", [])
        product_target_concerns = product.get("target_concerns", [])

        recommendation_type = opportunity.get("recommendation_type", "")
        base_rationale = opportunity.get("rationale", "")

        # Build context for LLM
        concerns_text = ", ".join(customer_concerns) if customer_concerns else "general skincare"
        ingredients_text = ", ".join(product_ingredients) if product_ingredients else "key active ingredients"
        target_concerns_text = ", ".join(product_target_concerns) if product_target_concerns else "various skin concerns"

        # Create prompt
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a soft-hearted, caring skincare expert and doctor with a genuine passion for helping people achieve their best skin.
            You have a warm, approachable personality and a knack for making skincare recommendations feel personal and thoughtful,
            like you're truly invested in each person's skincare journey.

Your task is to write a brief, natural, and engaging rationale (2-3 sentences) explaining why a specific product is recommended for a customer.
Write as if you're speaking directly to them with care and expertise.

Guidelines:
- Be warm, empathetic, and genuinely caring - like a doctor who truly wants to help
- Use a conversational, friendly tone that feels personal and supportive
- Reference the customer's specific skin concerns with understanding and compassion
- Highlight how the product addresses their needs in a way that feels tailored to them
- Mention key ingredients or benefits naturally, as if explaining to a friend
- Keep it concise (2-3 sentences max) but make every word count
- Don't use markdown or special formatting
- Focus on the customer's benefit and how this product fits into their unique skincare journey
- Show that you understand their needs and genuinely want to help them achieve their skincare goals"""),
            ("human", """You're recommending {product_name} to {customer_name}, and you want to explain why this product is perfect for them in a warm, caring way.

Here's what you know about {customer_name}:
- They're dealing with: {concerns_text}
- They're a {customer_tier} tier customer
- They've invested ${customer_ltv:.2f} in their skincare journey so far

About the product:
- It's a {product_category} priced at ${product_price:.2f}
- Key ingredients: {ingredients_text}
- It specifically targets: {target_concerns_text}

Why you're recommending it: {recommendation_type}
Context: {base_rationale}

Write a brief, warm, and personalized rationale (2-3 sentences) as if you're speaking directly to {customer_name} with genuine care and expertise.
Make them feel understood and supported in their skincare journey.""")
        ])

        # Initialize LLM
        llm = ChatOpenAI(
            model=config.llm_model,
            temperature=config.temperature
        )

        # Generate rationale
        chain = prompt | llm
        response = chain.invoke({
            "customer_name": customer_name,
            "concerns_text": concerns_text,
            "customer_tier": customer_tier,
            "customer_ltv": customer_ltv,
            "product_name": product_name,
            "product_category": product_category,
            "product_price": product_price,
            "ingredients_text": ingredients_text,
            "target_concerns_text": target_concerns_text,
            "recommendation_type": recommendation_type,
            "base_rationale": base_rationale
        })

        # Extract text from response
        rationale = response.content.strip()

        # Validate response (should be non-empty)
        if rationale and len(rationale) > 20:  # Minimum length check
            return rationale
        else:
            return None  # Fallback to rule-based

    except Exception as e:
        # Log error but don't fail - fallback to rule-based rationale
        print(f"Warning: LLM rationale generation failed: {str(e)}")
        return None


def enhance_opportunities_with_llm(
    opportunities: list[Dict[str, Any]],
    customer_data: Dict[str, Any],
    product_lookup: Dict[str, Dict[str, Any]],
    config: Tier2CrossSellUpsellConfig,
    max_opportunities: int = 3
) -> list[Dict[str, Any]]:
    """
    Enhance top opportunities with LLM-generated rationales.

    This function takes the top N opportunities and enhances their rationales
    with personalized, natural language explanations from an LLM.

    Args:
        opportunities: List of opportunity dicts (should be pre-sorted by score)
        customer_data: Customer data dict
        product_lookup: Dictionary mapping product_id to product dict
        config: Configuration with LLM settings
        max_opportunities: Maximum number of opportunities to enhance (cost control)

    Returns:
        List of opportunities with enhanced rationales (or original if LLM fails)
    """
    enhanced = []

    # Only enhance top N opportunities (cost control)
    opportunities_to_enhance = opportunities[:max_opportunities]

    for opportunity in opportunities:
        product_id = opportunity.get("product_id")
        product = product_lookup.get(product_id)

        if not product:
            enhanced.append(opportunity)
            continue

        # Enhance if in top N
        if opportunity in opportunities_to_enhance:
            llm_rationale = generate_llm_rationale(
                opportunity,
                customer_data,
                product,
                config
            )

            if llm_rationale:
                # Use LLM rationale, but keep original as fallback
                enhanced_opp = opportunity.copy()
                enhanced_opp["rationale"] = llm_rationale
                enhanced_opp["rationale_source"] = "llm"  # Track source
                enhanced.append(enhanced_opp)
            else:
                # Fallback to rule-based
                enhanced_opp = opportunity.copy()
                enhanced_opp["rationale_source"] = "rule_based"
                enhanced.append(enhanced_opp)
        else:
            # Keep original for opportunities beyond top N
            enhanced_opp = opportunity.copy()
            enhanced_opp["rationale_source"] = "rule_based"
            enhanced.append(enhanced_opp)

    return enhanced





## Updated persona

**Before:** Generic "skincare expert" with professional tone  
**After:** "Soft-hearted, caring skincare expert and doctor" with:
- Warm, empathetic, genuinely caring tone
- Conversational and friendly, like speaking to a friend
- Understanding and compassion for skin concerns
- Personal, supportive recommendations
- Genuine investment in the customer's skincare journey

## What this will do

The rationales will sound more like:
- "I understand you're dealing with hyperpigmentation, and I think this Vitamin C serum could really help brighten and even out your skin tone..."
- "Since you're missing the SPF step in your routine, I'd love to recommend this mattifying fluid that will protect your skin while addressing your acne concerns..."

Instead of:
- "Customer missing routine step 5. Product targets acne concerns."

## Ready to test

The LLM rationale generation is integrated and ready. When you run the orchestrator, the top 3 opportunities will get personalized, warm rationales from this persona.



# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_030_CrossSell_Upsell_Orchestrator % cd /Users/micahshull/Documents/AI_LangGraph/LG_Cursor_030_CrossSell_Upsell_Orchestrator && python3 -m agents.tier2_crosssell_upsell.run_orchestrator C003
============================================================
Running Tier 2 Cross-Sell & Upsell Orchestrator
Customer ID: C003
============================================================

Running orchestrator workflow...
/Users/micahshull/Documents/AI_LangGraph/LG_Cursor_030_CrossSell_Upsell_Orchestrator/.venv/lib/python3.13/site-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)

✅ Orchestrator completed successfully!

📊 Results Summary:
   Customer: Emily Chen (C003)
   Total opportunities: 11
   Top recommendations: 3
   Total potential revenue: $399.0
   Routine completeness: 20.0%

🏆 Top Recommendations:
   1. Radiance Boost Vitamin C Serum
      Score: 6.19/10.0 | Price: $44.00
      Type: replenishment
      Rationale: Emily, I truly understand how frustrating hyperpigmentation can be, and that's why I think the Radiance Boost Vitamin C Serum is such a wonderful fit for you. With its powerful blend of Vitamin C and Ferulic Acid, this serum not only targets those stubborn dark spots but also brightens your complexion beautifully. Since it's been a bit since your last purchase, replenishing this gem will help keep your skin glowing and on track toward your goals!

   2. UltraShield SPF 40
      Score: 5.22/10.0 | Price: $35.00
      Type: ingredient_match + routine_step_gap
      Rationale: Emily, I truly understand how frustrating hyperpigmentation can be, and that's why I think UltraShield SPF 40 would be a wonderful addition to your routine. With its powerful Zinc Oxide and nourishing Vitamin E, it not only protects your skin from sun damage but also helps to fade those dark spots over time. By incorporating this into your daily regimen, you’ll be taking a big step towards achieving that even, radiant complexion you’re aiming for!

   3. Firm+ Overnight Peptide Cream
      Score: 4.24/10.0 | Price: $52.00
      Type: routine_step_gap
      Rationale: Emily, I truly understand your concerns with hyperpigmentation, and I think the Firm+ Overnight Peptide Cream would be a wonderful addition to your routine. Packed with peptides and retinal, it not only helps to smooth fine lines but also works overnight to brighten your skin and even out your tone. This gentle yet effective moisturizer will complement your existing steps beautifully, helping you feel more confident in your skin as you continue your journey.

📄 Report saved to: output/tier2_cross_sell_reports/tier2_recommendations_C003_20251124_172015.md

============================================================


Reviewing the generated report:

The LLM rationales are working well. Here's what stands out:

## What's working

1. Personal and empathetic
   - "Emily, I truly understand how frustrating hyperpigmentation can be..."
   - Uses the customer's name and acknowledges their concern

2. Natural, conversational tone
   - "such a wonderful fit for you"
   - "this gem"
   - "big step towards achieving"

3. Product benefits explained clearly
   - Mentions specific ingredients (Vitamin C, Ferulic Acid, Zinc Oxide)
   - Connects benefits to the customer's concern

4. Encouraging and supportive
   - "help keep your skin glowing and on track toward your goals"
   - "helping you feel more confident in your skin as you continue your journey"

5. Context-aware
   - #1 mentions replenishment timing
   - #2 addresses routine gap + ingredient match
   - #3 explains how it complements existing steps

## Minor improvement opportunity

The opening phrase is similar across all three:
- "Emily, I truly understand how frustrating hyperpigmentation can be..."

Consider varying the opening to avoid repetition while keeping the warm tone.

## Comparison

Before (rule-based):
> "Customer missing routine step 5. Completes routine sequence. | Product targets acne concerns which match customer's skin concerns."

After (LLM):
> "Emily, I truly understand how frustrating hyperpigmentation can be, and that's why I think UltraShield SPF 40 would be a wonderful addition to your routine..."

## Overall assessment

The persona is working as intended. The rationales are:
- More engaging
- More personal
- More empathetic
- More actionable

The orchestrator now combines rule-based logic with personalized LLM-generated explanations. This should improve customer engagement and conversion.

Should I add variation to the opening phrases, or keep it as is?

Updating the prompt to encourage varied openings while keeping the warm, empathetic tone:

```
- **IMPORTANT: Vary your opening phrases** - don't start every recommendation the same way. Use different empathetic approaches:
  * Sometimes acknowledge their concern: "I know hyperpigmentation can be challenging..."
  * Sometimes focus on their journey: "As you continue working on your skincare routine..."
  * Sometimes highlight the opportunity: "I think this could be a game-changer for you..."
  * Sometimes be encouraging: "You're doing great with your routine, and adding this..."
  * Sometimes be direct but warm: "This product is perfect for addressing your hyperpigmentation..."
```

Updated the prompt to encourage varied openings while keeping the warm, empathetic tone. Changes:

## Updates made

1. Added explicit instruction to vary opening phrases with examples:
   - Acknowledge their concern: "I know hyperpigmentation can be challenging..."
   - Focus on their journey: "As you continue working on your skincare routine..."
   - Highlight the opportunity: "I think this could be a game-changer for you..."
   - Be encouraging: "You're doing great with your routine, and adding this..."
   - Be direct but warm: "This product is perfect for addressing your hyperpigmentation..."

2. Reinforced in the human prompt: added a note to vary the opening approach.

## Expected results

Instead of all three starting with "Emily, I truly understand how frustrating hyperpigmentation can be...", you'll get:
- More varied openings
- Same warm, empathetic tone
- More engaging and less repetitive
- Better suited for women customers

The LLM will naturally vary the openings while maintaining the caring doctor persona.

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_030_CrossSell_Upsell_Orchestrator % python3 -m agents.tier2_crosssell_upsell.run_orchestrator C003
============================================================
Running Tier 2 Cross-Sell & Upsell Orchestrator
Customer ID: C003
============================================================

Running orchestrator workflow...
/Users/micahshull/Documents/AI_LangGraph/LG_Cursor_030_CrossSell_Upsell_Orchestrator/.venv/lib/python3.13/site-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)

✅ Orchestrator completed successfully!

📊 Results Summary:
   Customer: Emily Chen (C003)
   Total opportunities: 11
   Top recommendations: 3
   Total potential revenue: $399.0
   Routine completeness: 20.0%

🏆 Top Recommendations:
   1. Radiance Boost Vitamin C Serum
      Score: 6.19/10.0 | Price: $44.00
      Type: replenishment
      Rationale: Emily, I know how frustrating hyperpigmentation can be, and I truly believe the Radiance Boost Vitamin C Serum is just what you need to continue your journey towards brighter skin. With its powerful combination of Vitamin C and Ferulic Acid, this serum not only targets those pesky dark spots but also helps to enhance your overall radiance. Since it's been a little while since your last purchase, replenishing this gem will keep your skin glowing and on track—you're doing wonderfully, and I’m here to support you every step of the way!

   2. UltraShield SPF 40
      Score: 5.22/10.0 | Price: $35.00
      Type: ingredient_match + routine_step_gap
      Rationale: Emily, I know how challenging hyperpigmentation can be, and I truly believe that adding UltraShield SPF 40 to your routine will make a significant difference. With its powerful Zinc Oxide and nourishing Vitamin E, this sunscreen not only protects your skin from sun damage but also helps to target those stubborn spots. You're doing such a great job investing in your skincare journey, and this step will really help you achieve the even-toned complexion you’re aiming for!

   3. Firm+ Overnight Peptide Cream
      Score: 4.24/10.0 | Price: $52.00
      Type: routine_step_gap
      Rationale: Emily, I see how dedicated you are to your skincare journey, and I truly believe that adding the Firm+ Overnight Peptide Cream could be a wonderful next step for you. This cream is packed with peptides and retinal, which not only help to combat fine lines but also work beautifully to improve your skin's overall texture and tone, addressing that hyperpigmentation you're concerned about. It’s like giving your skin a nourishing hug while you sleep, and I think you’ll love the results!

📄 Report saved to: output/tier2_cross_sell_reports/tier2_recommendations_C003_20251124_172624.md

============================================================
(.venv) micahshull@Micahs-iMac LG_Cursor_030_CrossSell_Upsell_Orchestrator %

The variation is working. Here's what changed:

## Improved variation

**Before (all similar):**
- "Emily, I truly understand how frustrating hyperpigmentation can be..."

**After (each unique):**
1. "Emily, I know how frustrating hyperpigmentation can be, and I truly believe the Radiance Boost Vitamin C Serum is just what you need to continue your journey towards brighter skin."
   - Focus: Journey and progress

2. "Emily, I know how challenging hyperpigmentation can be, and I truly believe that adding UltraShield SPF 40 to your routine will make a significant difference."
   - Focus: Impact and results

3. "Emily, I see how dedicated you are to your skincare journey, and I truly believe that adding the Firm+ Overnight Peptide Cream could be a wonderful next step for you."
   - Focus: Recognition of dedication

## What's working

- Each opening is unique while maintaining warmth
- Different angles: journey, impact, dedication
- Still empathetic and personal
- Natural flow without repetition
- The "nourishing hug" phrase in #3 adds a personal touch

The warm, doctor-like persona is working well, and the variation keeps each recommendation fresh and engaging. The orchestrator now combines:
- Rule-based logic (scoring, detection)
- LLM-powered personalization (warm, varied rationales)
- Production-ready architecture (error handling, fallbacks)



Yes—this aligns with the approach we've been following. Here's why it works:

## Why this approach is effective

### 1. MVP first (rule-based foundation)
- Fast to build: deterministic logic, no API dependencies
- Reliable: predictable, testable, debuggable
- Cost-effective: no LLM costs during development
- Solid foundation: architecture and patterns established

### 2. Fine-tuning layer (LLM enhancement)
- Iterate without breaking core logic
- A/B test different personas, tones, and styles
- Cost-controlled: only enhance top opportunities
- Fallback safety: if LLM fails, rule-based rationale still works

## The pattern we followed

This matches the Orchestrator Agents Guide pattern:

> "Rule-based first, LLM later" - Start with deterministic rules for speed, reliability, and cost. Add LLMs only where they add clear value.

## What this enables

1. Business value early: MVP delivers value while you refine
2. Risk mitigation: core logic is stable; LLM is additive
3. Continuous improvement: iterate on personalization without architectural changes
4. Cost optimization: use LLMs selectively (top 3 opportunities)
5. Team confidence: solid foundation before adding complexity

## Real-world impact

- Week 1: Working orchestrator (rule-based)
- Week 2: LLM personalization added
- Ongoing: Fine-tune tone, variation, persona based on customer feedback

This is faster and lower risk than building everything with LLMs from day one.

## The key insight

The architecture separates:
- Orchestration (nodes) = stable, rule-based
- Business logic (utilities) = rule-based, testable
- Personalization (LLM) = iterative, fine-tunable

This separation lets you refine the customer-facing layer without touching core logic.

This is a solid pattern for building production AI agents. You've built something reliable that can evolve based on real customer feedback.